In [5]:
import requests
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta

load_dotenv()

key = os.getenv("API_KEY")
location = "Bandung"
start_date = datetime(2023, 12, 1)  # Start date
end_date = datetime(2024, 12, 1)    # End date (12 months)

In [6]:
def fetch_weather_data(api_key, location, start_date, end_date):
    results = []
    current_start = start_date

    while current_start < end_date:
        current_end = min(current_start + timedelta(days=29), end_date)  # Ensure we don't exceed 30 days
        formatted_start = current_start.strftime("%Y-%m-%d")
        formatted_end = current_end.strftime("%Y-%m-%d")
        
        url = f'http://api.weatherapi.com/v1/history.json?key={api_key}&q={location}&dt={formatted_start}&end_dt={formatted_end}'
        response = requests.get(url)

        if response.status_code == 200:
            results.append(response.json())
        else:
            print(f"Failed to fetch data for {formatted_start} to {formatted_end}: {response.status_code}")
        
        current_start = current_end + timedelta(days=1)  # Move to the next period

    return results

# Fetch data
weather_data = fetch_weather_data(key, location, start_date, end_date)

# Output the results (e.g., length of data chunks fetched)
print(f"Fetched data for {len(weather_data)} periods.")


Failed to fetch data for 2023-12-01 to 2023-12-30: 400
Fetched data for 12 periods.


In [8]:
print(weather_data[1]['forecast']['forecastday'][0]['hour'][0]['temp_c'])

25.0


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
days = len(weather_data[0]['forecast']['forecastday'])
hours = len(weather_data[0]['forecast']['forecastday'][0]['hour'])

In [12]:
df = []

for month in range(len(weather_data)):
    days = len(weather_data[month]['forecast']['forecastday'])
    hours = len(weather_data[month]['forecast']['forecastday'][0]['hour'])
    for day in range(days):
        for hour in range(hours):
            field = {
                'time': weather_data[month]['forecast']['forecastday'][day]['hour'][hour]['time'],
                'temp': weather_data[month]['forecast']['forecastday'][day]['hour'][hour]['temp_c'],
                'wind_speed': weather_data[month]['forecast']['forecastday'][day]['hour'][hour]['wind_kph'],
                'wind_degree': weather_data[month]['forecast']['forecastday'][day]['hour'][hour]['wind_degree'],
                # 'wind_dir': weather_data[month]['forecast']['forecastday'][day]['hour'][hour]['wind_dir'],
                'pressure': weather_data[month]['forecast']['forecastday'][day]['hour'][hour]['pressure_in'],
                'precip': weather_data[month]['forecast']['forecastday'][day]['hour'][hour]['precip_in'],
                'humidity': weather_data[month]['forecast']['forecastday'][day]['hour'][hour]['humidity'],
                'cloud': weather_data[month]['forecast']['forecastday'][day]['hour'][hour]['cloud'],
                'uv': weather_data[month]['forecast']['forecastday'][day]['hour'][hour]['uv'],
                'weather': weather_data[month]['forecast']['forecastday'][day]['hour'][hour]['condition']['code'],
            }        
            
            df.append(field)

df = pd.DataFrame(df)
print(df)

                  time  temp  wind_speed  wind_degree  pressure  precip  \
0     2023-12-31 00:00  26.9         2.2          225     29.82    0.00   
1     2023-12-31 01:00  26.7         2.5          240     29.81    0.00   
2     2023-12-31 02:00  26.6         2.9          255     29.79    0.01   
3     2023-12-31 03:00  26.4         3.2          233     29.79    0.01   
4     2023-12-31 04:00  26.3         3.6          218     29.79    0.00   
...                ...   ...         ...          ...       ...     ...   
8083  2024-12-01 19:00  26.8         3.2           65     29.76    0.02   
8084  2024-12-01 20:00  26.8         2.2           86     29.78    0.03   
8085  2024-12-01 21:00  26.7         2.2          107     29.79    0.00   
8086  2024-12-01 22:00  26.6         1.8          136     29.79    0.00   
8087  2024-12-01 23:00  26.5         2.5          168     29.78    0.00   

      humidity  cloud   uv  weather  
0           79     42  0.0     1003  
1           80     32  

In [13]:
df.to_csv('weather_data.csv', index=False)